<a href="https://colab.research.google.com/github/saurabhIU/Deep-Learning/blob/master/Speech_Denoising_Using_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import glob
import glob
import librosa
import os

## Mount Google Drive on Colab

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!ls /content/gdrive/'My Drive/Assignment 3'

te  tr	v


In [5]:
tf.__version__

'1.13.1'

## Load  Data and get STFT

In [0]:
def load_files(path):
  
  file_td , file_fd , file_fd_abs, length = [] , [] , [], []
  
  for file in sorted(glob.iglob(path + '*.wav')):
    
    s,sr = librosa.load(file, sr=None)
    
    file_td.append(s)
    
    
    S = librosa.stft(s, n_fft=1024, hop_length=512)
    file_fd.append(S)
    
    S_abs = np.abs(S)
    padd_length = 200-S_abs.shape[1]
    S_abs  = np.pad(S_abs , ((0,0),(0, padd_length )), 'constant')
    file_fd_abs.append(S_abs)
    
    length.append(S_abs.shape[1])
    
  return file_td , file_fd , file_fd_abs, length
  

## Import Training files

In [7]:
# Training signal files

file_path = '/content/gdrive/My Drive/Assignment 3/tr/trx'
file_td , file_fd , file_fd_abs, length = load_files(file_path)

print("Training file download complete..")

# Training clean signal files

file_path = '/content/gdrive/My Drive/Assignment 3/tr/trs'
file_td_clean , file_fd_clean , file_fd_abs_clean, length_clean = load_files(file_path)
print("Clean signal file download complete..")

# Training Noise signal files

file_path = '/content/gdrive/My Drive/Assignment 3/tr/trn'
file_td_noise , file_fd_noise , file_fd_abs_noise, length_noise = load_files(file_path)
print("Noise file download complete..")

Training file download complete..
Clean signal file download complete..
Noise file download complete..


## Import Validation files

In [8]:
 # Validation signal files

file_path = '/content/gdrive/My Drive/Assignment 3/v/vx'
val_td , val_fd , val_fd_abs, val_length = load_files(file_path)

print("Validation training file download complete..")

# Validation clean signal files

file_path = '/content/gdrive/My Drive/Assignment 3/v/vs'
val_td_clean , val_fd_clean , val_fd_abs_clean, val_length_clean = load_files(file_path)
print("Validation Clean signal file download complete..")

# Validation Noise signal files

file_path = '/content/gdrive/My Drive/Assignment 3/v/vn'
val_td_noise , val_fd_noise , val_fd_abs_noise, val_length_noise = load_files(file_path)
print("Validation Noise file download complete..")

Validation training file download complete..
Validation Clean signal file download complete..
Validation Noise file download complete..


## Import Test Files

In [9]:
file_path = '/content/gdrive/My Drive/Assignment 3/te/'
test_td, test_fd, test_fd_abs, test_len = load_files(file_path)
print("Test file download complete..")

Test file download complete..


In [0]:
file_fd_abs_clean_ = file_fd_abs_clean[0:1023]
file_fd_abs_noise_ = file_fd_abs_noise[0:1023]

## Construct IBM Matrix

In [0]:
IBM_Matrix = [(file_fd_abs_clean_[i] > file_fd_abs_noise_[i]) for i in range(len(file_fd_abs_clean_))]

In [0]:
IBM_Matrix = [IBM_Matrix[i].astype(int) for i in range(len(IBM_Matrix))]


## RNN Model

In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(512, activation='relu', return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.LSTM(512, activation='relu', return_sequences=True))
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Dense(513, activation='relu'))

model.add(tf.keras.layers.Activation(tf.nn.sigmoid))
model.add(tf.keras.layers.Activation(tf.nn.relu))



## Reshape IBM and training matricies 

In [0]:
IBM = np.array(IBM_Matrix).swapaxes(1,2)

In [21]:
file_fd_abs_ = np.array(file_fd_abs).swapaxes(1,2)
print(file_fd_abs_.shape)

(1023, 200, 513)


In [22]:
print(IBM.shape)

(1023, 200, 513)


In [24]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

model.fit(file_fd_abs_,
          IBM,
          epochs=3,
          batch_size=10)

Epoch 1/3
1023/1023 [==============================] - 130s 128ms/sample - loss: 0.2603 - acc: 0.4324
Epoch 2/3
1023/1023 [==============================] - 129s 126ms/sample - loss: 0.2500 - acc: 0.7058
Epoch 3/3
1023/1023 [==============================] - 130s 127ms/sample - loss: 0.2500 - acc: 0.7058


In [27]:
file_fd_abs_[0].shape[1]

513

## SNR for Training data

In [31]:
file_fd_abs_.shape

(1023, 200, 513)

In [37]:
file_fd_abs_clean_ = file_fd_abs_clean[0:1023]
file_fd_abs_clean_ = np.array(file_fd_abs_clean_).swapaxes(1,2)
file_fd_abs_clean_.shape

(1023, 200, 513)

In [0]:
training_snr = []
for i in range(len(file_fd_abs_)):
  
    SNR_i = 0

    x = np.zeros((1, file_fd_abs_[i].shape[0], file_fd_abs_[i].shape[1]))
    y = np.zeros((1, file_fd_abs_clean_[i].shape[0], file_fd_abs_clean_[i].shape[1]))
    
    x[0,:,:] = file_fd_abs_[i]
    y[0,:,:] = file_fd_abs_clean_[i]

    
    prediction = model.predict(x,y)
            
    X = file_fd[i]
    s = file_td_clean[i]
    prediction = 1 * (prediction > 0.5)
    prediction = prediction.T
    S_prediction = prediction * X
    s_prediction = librosa.istft(S_prediction, win_length = 1024, hop_length = 512)
    s_clean = s[0:s_prediction.size]

    SNR_i = 10*np.log10(np.dot(s_clean.T,s_clean)/np.dot((s_clean - s_pred).T,(s_clean - s_pred)))
    training_snr.append(SNR_i)
    